In [ ]:
%pip install torch zkstats==0.1.10

In [ ]:
import json
import os

from zkstats.core import (
    prover_gen_settings,
    prover_gen_proof,
    setup,
    verifier_verify,
)

cwd = os.getcwd()

# FIXME: fill this in with the path to your data
data_path = f"{cwd}/data.json"

# Paths to the output files
output_dir = f"{cwd}/out"
os.makedirs(output_dir, exist_ok=True)
model_onnx_path = f"{output_dir}/model.onnx"
compiled_model_path = f"{output_dir}/model.compiled"

pk_path = f"{output_dir}/model.pk"
vk_path = f"{output_dir}/model.vk"
proof_path = f"{output_dir}/model.pf"
settings_path = f"{output_dir}/settings.json"
witness_path = f"{output_dir}/witness.json"
comb_data_path = f"{output_dir}/comb_data.json"
precal_witness_path = f"{output_dir}/precal_witness.json"

In [ ]:
# Data provided shared by the data provider beforehand

def get_data_shape(data_path: str) -> dict[str, int]:
    """
    Get the shape of the data from the data file.

    Parameters:
    - data_path (str): The path to the data file.

    Returns:
    - shape_info (dict): A dictionary where keys are column names and values are the number of elements (shape).
    """
    with open(data_path, 'r') as f:
        data = json.load(f)
    shape_info = {col: len(data[col]) for col in data.keys()}
    return shape_info

data_shape = get_data_shape(data_path)
print(data_shape)
data_commitment_path = f"{output_dir}/data_commitment.json"

## User-defined Computation

A computation should be of type `TComputation`. For example, the following code snippet defines a computation that computes the sum of the private data.

```python
def computation(state: State, args: Args):
    x = args["x"]
    y = args["y"]
    return state.mean(x), state.mean(y)
```

FIXME: The following code snippet is entirely from the user. You MUST check
1. the code only performs zkstats-related operations.
2. the computation must not leak any information about the private data.

In [ ]:
# This is just a dummy computation. Replace it with user's computation
import torch
from zkstats.computation import State, Args

def computation(state: State, args: Args):
    x = args["x"]
    y = args["y"]
    return state.mean(x), state.mean(y)

Generate settings and setup with user's computation.

In [ ]:
from zkstats.computation import computation_to_model

used_columns, state, model = computation_to_model(computation, precal_witness_path, data_shape, isProver=True)
prover_gen_settings(
    data_path,
    used_columns,
    comb_data_path,
    model,
    model_onnx_path,
    [7],
    "resources",
    settings_path,
)

# Determine which srs to use with the logrows in the settings.json
with open(settings_path, "r") as f:
    settings = json.load(f)
logrows = int(settings["run_args"]["logrows"])
srs_path = f'~/.ezkl/srs/kzg{logrows}.srs'

In [ ]:
setup(model_onnx_path, compiled_model_path, settings_path, vk_path, pk_path)

Generate proof with your data and user's computation.

In [ ]:
prover_gen_proof(
    model_onnx_path,
    comb_data_path,
    witness_path,
    compiled_model_path,
    settings_path,
    proof_path,
    pk_path,
)

## Verify the proof to ensure it is correct
NOTE: The following section is to illustrate what should be done on the user (data consumer) side. This step is not required by the data provider.

In [ ]:
# verify the proof
res = verifier_verify(proof_path, settings_path, vk_path, used_columns, data_commitment_path)
print(f"!@# res: {res}")


Print the file paths. You should share the following files back to the user for them to verify the proof. You **SHOULD NOT** share more files otherwise data might be leaked.

In [ ]:
print("Model onnx:\t\t", model_onnx_path)
print("Settings:\t\t", settings_path)
print("Proof:\t\t\t", proof_path)
print("Verification key:\t", vk_path)
print("Srs path:\t\t", srs_path)